In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import shutil
src_dirs =['/content/drive/MyDrive/image clustering/efficient_image_clustering/source code/blured military data',
           '/content/drive/MyDrive/image clustering/efficient_image_clustering/source code/low contrast military image',
           '/content/drive/MyDrive/image clustering/efficient_image_clustering/source code/noisy military data',
           '/content/drive/MyDrive/image clustering/efficient_image_clustering/source code/original military image']
dest_dir = '/content/drive/MyDrive/image clustering/efficient_image_clustering/source code/dataset'
for dir in src_dirs:
  files = os.listdir(dir)
  for file in files:
    src = os.path.join(dir , file)
    dest = os.path.join(dest_dir , file)
    shutil.copy(src , dest)


In [ ]:
dir = '/content/drive/MyDrive/image clustering/efficient_image_clustering/source code/dataset/'
import os
files = os.listdir(dir)
import pandas as pd
df = pd.DataFrame({
    'image':[dir+file for file in files],
})

In [ ]:
def class_column(image):
  classes = ['blurred' , 'noise' , 'contrast']
  for cls in classes:
    if cls in str(image):
      return 'non-sharp'
  return 'sharp'
df['class'] = df['image'].apply(class_column)

In [ ]:
df.head()

,image,class
0,/content/drive/MyDrive/image clustering/effici...,non-sharp
1,/content/drive/MyDrive/image clustering/effici...,non-sharp
2,/content/drive/MyDrive/image clustering/effici...,non-sharp
3,/content/drive/MyDrive/image clustering/effici...,non-sharp
4,/content/drive/MyDrive/image clustering/effici...,non-sharp


In [ ]:
import os
dir = [file for file in os.listdir('/content/drive/MyDrive/image clustering/efficient_image_clustering/source code/dataset') if 'noise' in str(file) ]
print(len(dir))

569


In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import os

# Define your source and destination directories
src_dir = '/content/drive/MyDrive/image clustering/efficient_image_clustering/source code/noisy military data'
dest_dir = '/content/drive/MyDrive/image clustering/efficient_image_clustering/source code/dataset'

# Create a data generator
datagen = ImageDataGenerator(
        rotation_range=0,
        width_shift_range=0,
        height_shift_range=0,
        shear_range=0,
        zoom_range=0.38, # Random zoom
        horizontal_flip=True, # Random horizontal flip
        vertical_flip=True, # Random vertical flip
        fill_mode='nearest')

# Loop over each image in the source directory
for filename in os.listdir(src_dir):
    img_path = os.path.join(src_dir, filename)
    img = load_img(img_path)  # this is a PIL image
    x = img_to_array(img)  # this is a Numpy array with shape (3, width, height)
    x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, width, height)

    # the .flow() command below generates batches of randomly transformed images
    # and saves the results to the `dest_dir` directory
    i = 0
    for batch in datagen.flow(x, batch_size=1,
                              save_to_dir=dest_dir, save_prefix='noise', save_format='jpeg'):
        i += 1
        if i > 7:
            break  # otherwise the generator would loop indefinitely


In [ ]:
len(os.listdir('/content/drive/MyDrive/image clustering/efficient_image_clustering/source code/low contrast military image'))

56

In [ ]:
dir = [file for file in os.listdir('/content/drive/MyDrive/image clustering/efficient_image_clustering/source code/dataset') if 'contrast' in file]
print(len(dir))

1397


In [ ]:
dir = '/content/drive/MyDrive/image clustering/efficient_image_clustering/source code/dataset/'
files = os.listdir('/content/drive/MyDrive/image clustering/efficient_image_clustering/source code/dataset')
import pandas as pd
df = pd.DataFrame({
    'image':[dir+file for file in files],
})
print(len(df['image'].to_list()))
def class_column(image):
  classes = ['blur' , 'noise' , 'contrast']
  for cls in classes:
    if cls in str(image):
      return 'non-sharp'
  return 'sharp'
df['class'] = df['image'].apply(class_column)

2174


In [ ]:
df.head()

,image,class
0,/content/drive/MyDrive/image clustering/effici...,non-sharp
1,/content/drive/MyDrive/image clustering/effici...,non-sharp
2,/content/drive/MyDrive/image clustering/effici...,non-sharp
3,/content/drive/MyDrive/image clustering/effici...,non-sharp
4,/content/drive/MyDrive/image clustering/effici...,non-sharp


In [ ]:
cnt = 0
print(df['class'].to_list().count('sharp'))

519


In [ ]:
df['image'].to_list()[-1]

'/content/drive/MyDrive/image clustering/efficient_image_clustering/source code/dataset/noise_0_8038.jpeg'

In [ ]:
df.to_csv(r'/content/drive/MyDrive/image clustering/efficient_image_clustering/source code/dataset.csv')

In [ ]:
import cv2
import torch
import torchvision.transforms as transforms
from torchvision.models import alexnet
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

# load the pre-trained AlexNet model
model = alexnet(pretrained=True)

# remove the last layer to get the features instead of predictions
model.classifier = model.classifier[:-1]

# define the image transformations
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# initialize an empty list to store the features
features = []

# loop over each image in the DataFrame
for img_path in df['image']:
    # read the image
    img = cv2.imread(img_path)

    # apply the transformations
    img = transform(img)

    # add an extra dimension
    img = img.unsqueeze(0)

    # extract the features
    feature = model(img)

    # append the features to the list
    features.append(feature.detach().numpy().flatten())

# convert the list of features to a DataFrame
features_df = pd.DataFrame(features)

# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_df, df['class'], test_size=0.2, random_state=42)

# train the SVC
svc = SVC()
svc.fit(X_train, y_train)

# print the accuracy on the test set
print('Test Accuracy:', svc.score(X_test, y_test))


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Test Accuracy: 0.8188908145580589


In [ ]:
from sklearn.metrics import classification_report

# predict the classes for the test set
y_pred = svc.predict(X_test)

# generate the classification report
report = classification_report(y_test, y_pred)

print('Classification Report:')
print(report)


Classification Report:
              precision    recall  f1-score   support

   non-sharp       0.85      0.84      0.85       677
       sharp       0.78      0.78      0.78       477

    accuracy                           0.82      1154
   macro avg       0.81      0.81      0.81      1154
weighted avg       0.82      0.82      0.82      1154



In [ ]:
import os
import pandas as pd
dir = '/content/drive/MyDrive/image clustering/efficient_image_clustering/source code/dataset'
files = os.listdir(dir)
original = [file for file in files if 'noise' not in str(file) and 'blur' not in str(file) and 'contrast' not in str(file)][:1000]
blur = [file for file in files if 'blur' in str(file)][:333]
contrast = [file for file in files if 'contrast' in str(file)][:333]
noise = [file for file in files if 'noise' in str(file)][:333]

# Create a list of tuples, where each tuple is a pair of (file, class)
data = [(dir+'/'+file, 'original') for file in original]
data += [(dir+'/'+file, 'not') for file in blur]
data += [(dir+'/'+file, 'not') for file in contrast]
data += [(dir+'/'+file, 'not') for file in noise]

# Convert the list of tuples into a DataFrame
df = pd.DataFrame(data, columns=['image', 'class'])

In [ ]:
df['class'].to_list().count('not')

999

In [ ]:
len(df['class'].to_list())

1999

In [ ]:
import pandas as pd
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing import image
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import classification_report

# Load pre-trained ResNet50 model + remove final classification layers
model = ResNet50(weights='imagenet', include_top=False)

def extract_features(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    features = model.predict(preprocessed_img)
    flattened_features = features.flatten()
    return flattened_features

# Assuming df is your DataFrame and it includes a column 'image_path'
features = []
for i in df['image']:
    features.append(extract_features(i, model))

# Add features to dataframe
df['features'] = features

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df['features'].tolist(), df['class'], test_size=0.2, random_state=42)

# Train the SVC
svc = SVC(kernel='linear')
svc.fit(X_train, y_train)

# predict the classes for the test set
y_pred = svc.predict(X_test)

# generate the classification report
report = classification_report(y_test, y_pred)

print('Classification Report:')
print(report)


1/1 [==============================] - 0s 213ms/step


In [ ]:
import pandas as pd
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import numpy as np

# Load pre-trained VGG16 model + remove final classification layers
model = VGG16(weights='imagenet', include_top=False)

def extract_features(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    features = model.predict(preprocessed_img)
    flattened_features = features.flatten()
    return flattened_features

# Assuming df is your DataFrame and it includes a column 'image_path'
features = []
for i in df['image']:
    features.append(extract_features(i, model))

# Add features to dataframe
df['features'] = features

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df['features'].tolist(), df['class'], test_size=0.2, random_state=42)

# Train the SVC
svc = SVC(kernel='linear')
svc.fit(X_train, y_train)


1/1 [==============================] - 1s 544ms/step


NameError: name 'classification_report' is not defined

In [ ]:
from sklearn.metrics import classification_report

# predict the classes for the test set
y_pred = svc.predict(X_test)

# generate the classification report
report = classification_report(y_test, y_pred)

print('Classification Report:')
print(report)

Classification Report:
              precision    recall  f1-score   support

         not       0.82      0.90      0.86       193
    original       0.89      0.82      0.86       207

    accuracy                           0.86       400
   macro avg       0.86      0.86      0.86       400
weighted avg       0.86      0.86      0.86       400



In [ ]:
import os
import pandas as pd
original = os.listdir('/content/drive/MyDrive/image clustering/original military image')[1453:3453]
blur = os.listdir('/content/drive/MyDrive/image clustering/blured military data')[1000:3000]
contrast = os.listdir('/content/drive/MyDrive/image clustering/low contrast military image')[1000:3000]
noise = os.listdir('/content/drive/MyDrive/image clustering/noisy military data')[5000:7000]

# Create a list of tuples, where each tuple is a pair of (file, class)
data = [('/content/drive/MyDrive/image clustering/original military image'+'/'+file, 'original') for file in original]
data += [('/content/drive/MyDrive/image clustering/blured military data'+'/'+file, 'blur') for file in blur]
data += [('/content/drive/MyDrive/image clustering/low contrast military image'+'/'+file, 'contrast') for file in contrast]
data += [('/content/drive/MyDrive/image clustering/noisy military data'+'/'+file, 'noise') for file in noise]

# Convert the list of tuples into a DataFrame
df = pd.DataFrame(data, columns=['image', 'class'])

In [ ]:
len(df['class'].to_list())

8000

In [ ]:
import pandas as pd
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import numpy as np

# Load pre-trained VGG16 model + remove final classification layers
model = VGG16(weights='imagenet', include_top=False)

def extract_features(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    features = model.predict(preprocessed_img)
    flattened_features = features.flatten()
    return flattened_features

# Assuming df is your DataFrame and it includes a column 'image_path'
features = []
for i in df['image']:
    features.append(extract_features(i, model))

# Add features to dataframe
df['features'] = features

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df['features'].tolist(), df['class'], test_size=0.2, random_state=42)

# Train the SVC
svc = SVC(kernel='linear')
svc.fit(X_train, y_train)

from sklearn.metrics import classification_report

# predict the classes for the test set
y_pred = svc.predict(X_test)

# generate the classification report
report = classification_report(y_test, y_pred)

print('Classification Report:')
print(report)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 1s 511ms/step


KeyboardInterrupt: 

In [ ]:
'''
import os
import shutil

src_dirs = ['/content/drive/MyDrive/image classification/efficient_image_clustering/source code/blured military data',
           '/content/drive/MyDrive/image classification/efficient_image_clustering/source code/low contrast military image',
           '/content/drive/MyDrive/image classification/efficient_image_clustering/source code/noisy military data',
           '/content/drive/MyDrive/image classification/efficient_image_clustering/source code/original military image'
           ]
dest_dir = '/content/drive/MyDrive/image classification/original_only'
os.makedirs(dest_dir , exist_ok=True)
for dir in src_dirs:
  for img in os.listdir(dir):
    src = os.path.join(dir , img)
    dest = os.path.join(dest_dir , img)
    shutil.copy(src , dest)
'''

In [ ]:
'''
directory = '/content/drive/MyDrive/image classification/blured military data'

for filename in os.listdir(directory):
    old_path = os.path.join(directory, filename)
    new_path = os.path.join(directory, 'noise_' + filename)
    os.rename(old_path, new_path)
'''

In [ ]:
'''
src_dir = '/content/drive/MyDrive/image classification/noisy military data'
for img in os.listdir(src_dir)[:403]:
  src = os.path.join(src_dir , img)
  dest = os.path.join(dest_dir , img)
  shutil.copy(src , dest)
'''

In [ ]:
import pandas as pd
def tag(name):
  if 'blur' in name:
    return 'blur'
  elif 'noise' in name:
    return 'noise'
  elif 'contrast' in name:
    return 'contrast'
  else:
    return 'sharp'
tup_list=[]
dest_dir = ''
for img in os.listdir(dest_dir):
  name = os.path.join(dest_dir , img)
  tup_list += [(name,tag(img))]
df = pd.DataFrame(tup_list , columns=['image' , 'class'])
df.to_csv('/content/drive/MyDrive/image classification/plz_be_last_dataset.csv')

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from sklearn import svm
import pandas as pd
import numpy as np
model = VGG16(weights='imagenet', include_top=False)
def extract_features(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = model.predict(x)
    return features.flatten()  # Flatten the features

df = pd.read_csv('/content/drive/MyDrive/image classification/plz_be_last_dataset.csv')  # replace with your dataframe

image_paths = df['image'].tolist()  # List of paths to your images
labels = df['class'].tolist()  # Corresponding labels

features = [extract_features(img_path) for img_path in image_paths]

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.2, random_state=7)

clf = svm.SVC()
clf.fit(features_train, labels_train)

labels_pred = clf.predict(features_test)

print(classification_report(labels_test, labels_pred))


1/1 [==============================] - 1s 855ms/step
              precision    recall  f1-score   support

        blur       0.99      0.95      0.97        99
    contrast       0.96      0.99      0.97        96
       noise       1.00      0.96      0.98       107
       sharp       0.94      0.98      0.96       114

    accuracy                           0.97       416
   macro avg       0.97      0.97      0.97       416
weighted avg       0.97      0.97      0.97       416



In [ ]:
import pickle

# Save the trained model as a pickle string.
saved_model = pickle.dumps(clf)

# Save the pickled model to a file
with open('/content/drive/MyDrive/image classification/saved_svc.pkl', 'wb') as file:
    pickle.dump(clf, file)

In [ ]:
import os
import glob
import shutil
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16, preprocess_input
import numpy as np
import pickle

# Load the pickled model
with open('/content/drive/MyDrive/image classification/saved_svc.pkl', 'rb') as file:
    clf_loaded = pickle.load(file)

# Load VGG16 model for feature extraction
model = VGG16(weights='imagenet', include_top=False)

def extract_features(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = model.predict(x)
    return features.flatten()

# Directory containing images
image_dir = '/content/drive/MyDrive/image classification/original_only'  # replace with your directory

# Directories to save images based on their predicted class
sharp_dir = '/content/drive/MyDrive/image_classification/sharp_images/'
other_dir = '/content/drive/MyDrive/image_classification/other_images/'

# Create directories if they don't exist
if not os.path.exists(sharp_dir):
    os.makedirs(sharp_dir)
if not os.path.exists(other_dir):
    os.makedirs(other_dir)

# Iterate over images in the directory
for img_path in glob.glob(os.path.join(image_dir, '*')):
    # Extract features and reshape them into the expected format
    features = extract_features(img_path).reshape(1, -1)

    # Use the loaded model to predict the class
    pred_class = clf_loaded.predict(features)[0]

    # Save the image in the appropriate directory based on its predicted class
    if pred_class == 'sharp':
        shutil.copy(img_path, sharp_dir)
    else:
        shutil.copy(img_path, other_dir)


1/1 [==============================] - 1s 565ms/step
